# Car Parts Detection - Colab 
Run this notebook in Google Colab with GPU runtime enabled.

**Setup:** Runtime → Change runtime type → GPU → Save

In [ ]:
# Install dependencies
%pip install ultralytics yt-dlp pyarrow -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.1/182.1 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 99.9 MB/s eta 0:00:00


In [2]:
# Verify GPU is available
import torch
print(f"GPU available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

GPU available: True
GPU: Tesla T4


## 1. Download and Extract Video Frames

In [ ]:
# Download video
!yt-dlp -f "bestvideo[ext=mp4]+bestaudio[ext=m4a]/best[ext=mp4]" \
  -o "input_video.mp4" \
  "https://www.youtube.com/watch?v=YcvECxtXoxQ"

[youtube] Extracting URL: https://www.youtube.com/watch?v=YcvECxtXoxQ
[youtube] YcvECxtXoxQ: Downloading webpage
[youtube] YcvECxtXoxQ: Downloading android vr player API JSON
[info] YcvECxtXoxQ: Downloading 1 format(s): 401+140
[download] Destination: input_video.f401.mp4
[download] 100% of    4.26GiB in 00:02:22 at 30.68MiB/s
[download] Destination: input_video.f140.m4a
[download] 100% of   43.12MiB in 00:00:02 at 21.30MiB/s
[Merger] Merging formats into "input_video.mp4"
Deleting original file input_video.f140.m4a (pass -k to keep)
Deleting original file input_video.f401.mp4 (pass -k to keep)


In [ ]:
# Extract 45-second clip starting at 2:00
!ffmpeg -ss 00:02:00 -i input_video.mp4 -t 45 -c copy clip_120_165.mp4 -y

# Extract frames (1 per second)
!mkdir -p frames
!ffmpeg -i clip_120_165.mp4 -vf "fps=1" frames/frame_%04d.jpg -y

# Verify
import os
print(f"Extracted {len([f for f in os.listdir('frames') if f.endswith('.jpg')])} frames")

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

## 2. Train Car Parts Segmentation Model

In [ ]:
from ultralytics import YOLO
import os

# Train model on car parts dataset (with GPU this takes ~15-30 min)
weights_path = "runs/segment/train/weights/best.pt"
if not os.path.exists(weights_path):
    print("Training car parts segmentation model...")
    base_model = YOLO("yolov8n-seg.pt")
    base_model.train(data="carparts-seg.yaml", epochs=50, imgsz=640)
    print("Training complete!")
else:
    print("Model already trained, loading weights...")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Training car parts segmentation model...
Ultralytics 8.4.14 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 14913MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=carparts-seg.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0

## 3. Run Detection on Video Frames

In [ ]:

from ultralytics import YOLO
import os, json

# Load trained car parts model
model = YOLO("runs/segment/train/weights/best.pt")

frames_dir = "frames"
results_list = []

for fname in sorted(os.listdir(frames_dir)):
    if not fname.endswith(".jpg"):
        continue

    frame_idx = int(fname.replace("frame_", "").replace(".jpg", ""))
    timestamp_sec = 120 + (frame_idx - 1)  # Clip starts at 2:00 (120 sec)

    path = os.path.join(frames_dir, fname)
    results = model(path)[0]

    for box in results.boxes:
        results_list.append({
            "video_id": "YcvECxtXoxQ",
            "frame_idx": frame_idx,
            "timestamp_sec": timestamp_sec,
            "class_label": model.names[int(box.cls)],
            "confidence_score": float(box.conf),
            "x_min": float(box.xyxy[0][0]),
            "y_min": float(box.xyxy[0][1]),
            "x_max": float(box.xyxy[0][2]),
            "y_max": float(box.xyxy[0][3])
        })

print(f"Total detections: {len(results_list)}")


image 1/1 /content/frames/frame_0001.jpg: 384x640 1 back_glass, 2 right_mirrors, 51.8ms
Speed: 2.2ms preprocess, 51.8ms inference, 47.5ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 /content/frames/frame_0002.jpg: 384x640 1 back_bumper, 1 back_glass, 1 back_left_light, 1 tailgate, 7.7ms
Speed: 2.8ms preprocess, 7.7ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 /content/frames/frame_0003.jpg: 384x640 2 back_glasss, 7.2ms
Speed: 2.0ms preprocess, 7.2ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 /content/frames/frame_0004.jpg: 384x640 1 back_bumper, 8.2ms
Speed: 1.9ms preprocess, 8.2ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 /content/frames/frame_0005.jpg: 384x640 1 back_glass, 7.1ms
Speed: 1.9ms preprocess, 7.1ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 /content/frames/frame_0006.jpg: 384x640 1 back_glass, 1 hood, 7.3ms
Speed: 1.9ms prepr

## 4. Save Detections to Parquet

In [ ]:
import pandas as pd

df = pd.DataFrame(results_list)
df.to_parquet("video_detections.parquet", index=False)
print(df.head(10))
print(f"\nUnique classes detected: {df['class_label'].unique()}")

      video_id  frame_idx  timestamp_sec      class_label  confidence_score  \
0  YcvECxtXoxQ          1            120       back_glass          0.477926   
1  YcvECxtXoxQ          1            120     right_mirror          0.251558   
2  YcvECxtXoxQ          1            120     right_mirror          0.250252   
3  YcvECxtXoxQ          2            121       back_glass          0.727625   
4  YcvECxtXoxQ          2            121         tailgate          0.563668   
5  YcvECxtXoxQ          2            121      back_bumper          0.410849   
6  YcvECxtXoxQ          2            121  back_left_light          0.364683   
7  YcvECxtXoxQ          3            122       back_glass          0.273512   
8  YcvECxtXoxQ          3            122       back_glass          0.261468   
9  YcvECxtXoxQ          4            123      back_bumper          0.253153   

         x_min        y_min        x_max        y_max  
0     4.049194    91.100807  2189.681396   789.070312  
1  1630.784546   9

## 5. Download Results

In [ ]:
from google.colab import files

# Download parquet file
files.download("video_detections.parquet")

# Download trained model weights
files.download("runs/segment/train/weights/best.pt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Image-to-Video Semantic Retrieval

In [1]:
# Step 1: Load detection index
import pandas as pd

df = pd.read_parquet("video_detections.parquet")
print(df.head())

      video_id  frame_idx  timestamp_sec   class_label  confidence_score  \
0  YcvECxtXoxQ          1            120    back_glass          0.477926   
1  YcvECxtXoxQ          1            120  right_mirror          0.251558   
2  YcvECxtXoxQ          1            120  right_mirror          0.250252   
3  YcvECxtXoxQ          2            121    back_glass          0.727625   
4  YcvECxtXoxQ          2            121      tailgate          0.563668   

         x_min        y_min        x_max        y_max  
0     4.049194    91.100807  2189.681396   789.070312  
1  1630.784546   969.553894  1793.621704  1147.091064  
2  1626.520386   909.701660  1788.607544  1127.106079  
3     7.212891    29.714424  2619.140625   654.623779  
4  2322.960449  1281.673218  3017.492188  1955.216309  


In [3]:
# Step 2: Load model and define query class detector
from ultralytics import YOLO

model = YOLO("best.pt")  # trained car parts model (local)

def get_query_classes(image_path, confidence_threshold=0.5):
    """Run detector on a query image, return detected class names above threshold."""
    results = model(image_path, verbose=False)[0]
    detected = []
    for box in results.boxes:
        if float(box.conf) >= confidence_threshold:
            detected.append(model.names[int(box.cls)])
    return list(set(detected))

print("Model ready. Classes:", list(model.names.values()))


Model ready. Classes: ['back_bumper', 'back_door', 'back_glass', 'back_left_door', 'back_left_light', 'back_light', 'back_right_door', 'back_right_light', 'front_bumper', 'front_door', 'front_glass', 'front_left_door', 'front_left_light', 'front_light', 'front_right_door', 'front_right_light', 'hood', 'left_mirror', 'object', 'right_mirror', 'tailgate', 'trunk', 'wheel']


In [4]:
# Step 3 & 4: Merge detections into contiguous time intervals
def merge_intervals(timestamps, gap_threshold=10):
    """Merge timestamps that are close together into contiguous intervals."""
    if len(timestamps) == 0:
        return []
    timestamps = sorted(timestamps)
    intervals = []
    start = timestamps[0]
    end = timestamps[0]
    for t in timestamps[1:]:
        if t - end <= gap_threshold:
            end = t
        else:
            intervals.append((start, end))
            start = t
            end = t
    intervals.append((start, end))
    return intervals

In [5]:
# Step 5 & 6: Run retrieval over all Hugging Face query images
%pip install datasets -q
from datasets import load_dataset

ds = load_dataset("aegean-ai/rav4-exterior-images", split="train")
all_results = []

for sample in ds:
    image = sample["image"]  # PIL Image
    image.save("temp_query.jpg")

    query_classes = get_query_classes("temp_query.jpg")
    if not query_classes:
        continue

    matches = df[df["class_label"].isin(query_classes)].copy()
    matches = matches.sort_values("timestamp_sec")
    intervals = merge_intervals(matches["timestamp_sec"].tolist())

    for (start, end) in intervals:
        segment_detections = matches[
            (matches["timestamp_sec"] >= start) & (matches["timestamp_sec"] <= end)
        ]
        all_results.append({
            "query_timestamp": sample["timestamp"],
            "start_timestamp": start,
            "end_timestamp": end,
            "class_label": ", ".join(query_classes),
            "number_of_supporting_detections": len(segment_detections),
            "verify_url": f"https://www.youtube.com/embed/YcvECxtXoxQ?start={start}&end={end}"
        })

results_df = pd.DataFrame(all_results)
results_df.to_parquet("retrieval_results.parquet", index=False)
print(f"Total retrieval results: {len(results_df)}")
print(results_df.head())

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Note: you may need to restart the kernel to use updated packages.


/opt/conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 100%|██████████| 65/65 [00:00<00:00, 65.39 examples/s]


Total retrieval results: 72
  query_timestamp  start_timestamp  end_timestamp  \
0           00:00              120            120   
1           00:00              131            165   
2           00:05              135            165   
3           00:10              135            165   
4           00:15              120            165   

                                         class_label  \
0  front_right_door, wheel, front_glass, front_le...   
1  front_right_door, wheel, front_glass, front_le...   
2  front_bumper, wheel, front_left_door, back_lef...   
3  front_left_door, back_left_door, front_bumper,...   
4  front_bumper, wheel, front_glass, hood, front_...   

   number_of_supporting_detections  \
0                                2   
1                               21   
2                               10   
3                               10   
4                               29   

                                          verify_url  
0  https://www.youtube.com/embed